In [22]:
import requests
import json
import os

# 目标 URL
url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=last-30-days&FORMAT=json'

# 本地保存路径
out_dir = 'tle_json'
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, 'last_30_days.json')

# 1. 发送请求
resp = requests.get(url)
resp.raise_for_status()  # 如果返回不是 200，会抛出异常

# 2. 解析为 Python 对象（列表/字典）
data = resp.json()

# 3. 写入本地文件
with open(out_path, 'w', encoding='utf-8') as f:
    # indent=2 保持良好可读性，ensure_ascii=False 保留 Unicode
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f'已保存到 {out_path}，共 {len(data)} 条记录')




已保存到 tle_json\last_30_days.json，共 163 条记录


In [24]:
#!/usr/bin/env python3
# compute_state_vectors.py

import json
import numpy as np
import pandas as pd
from astropy import units as u
from astropy.time import Time
from poliastro.bodies import Earth
from poliastro.twobody.orbit import Orbit

# 输入 JSON 文件路径
INPUT_JSON = 'tle_json/last_30_days.json'
# 输出 CSV 文件路径
OUTPUT_CSV = 'last_30_days_state_vectors_corrected.csv'

def main():
    # 1. 读取 GP JSON 数据
    with open(INPUT_JSON, 'r', encoding='utf-8') as f:
        sats = json.load(f)

    records = []

    for sat in sats:
        name = sat.get('OBJECT_NAME', 'UNKNOWN')
        epoch_str = sat['EPOCH']  # ISO 格式时间字符串
        epoch = Time(epoch_str, format='isot', scale='utc')

        # 2. 经典轨道要素（OMM JSON 字段）
        inc  = sat['INCLINATION']        * u.deg
        raan = sat['RA_OF_ASC_NODE']     * u.deg
        ecc  = sat['ECCENTRICITY']       * u.one
        argp = sat['ARG_OF_PERICENTER']  * u.deg
        M    = sat['MEAN_ANOMALY']       * u.deg

        # 3. 平均运动从 rev/day → rad/s
        #    sat['MEAN_MOTION'] 单位为 rev/day
        mean_motion_rev_per_day = sat['MEAN_MOTION'] * u.one / u.day
        # 转换到 1/s，再乘 2π 得到 rad/s
        n_rad_s = mean_motion_rev_per_day.to(1 / u.s) * (2 * np.pi)

        # 4. 根据开普勒第三定律求半长轴 a（km）
        mu = Earth.k.to(u.km**3 / u.s**2)
        a = (mu / n_rad_s**2) ** (1/3)

        # 5. 求偏近心角 E，再计算真近点角 ν
        M_rad = M.to(u.rad).value
        e = ecc.value
        E = M_rad  # 初始猜测
        for _ in range(100):
            E = E - (E - e * np.sin(E) - M_rad) / (1 - e * np.cos(E))
        nu = 2 * np.arctan2(
            np.sqrt(1 + e) * np.sin(E / 2),
            np.sqrt(1 - e) * np.cos(E / 2)
        ) * u.rad

        # 6. 构造 Orbit 并提取状态向量
        orb = Orbit.from_classical(
            attractor=Earth,
            a=a,
            ecc=ecc,
            inc=inc,
            raan=raan,
            argp=argp,
            nu=nu,
            epoch=epoch
        )
        r_vec = orb.r.to(u.km).value   # [x, y, z] km
        v_vec = orb.v.to(u.km / u.s).value  # [vx, vy, vz] km/s

        records.append({
            'OBJECT_NAME': name,
            'EPOCH':       epoch_str,
            'x_km':        r_vec[0],
            'y_km':        r_vec[1],
            'z_km':        r_vec[2],
            'vx_km_s':     v_vec[0],
            'vy_km_s':     v_vec[1],
            'vz_km_s':     v_vec[2],
        })

    # 7. 保存结果
    df = pd.DataFrame(records)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f'✅ 已生成 {len(df)} 条状态向量，保存在 {OUTPUT_CSV}')

if __name__ == '__main__':
    main()

print(df.head())


✅ 已生成 163 条状态向量，保存在 last_30_days_state_vectors_corrected.csv
     OBJECT_NAME  OBJECT_ID                       EPOCH  MEAN_MOTION  \
0    APRIZESAT 2  2004-025A  2025-05-04T01:11:26.933856    14.377811   
1  SAUDICOMSAT 1  2004-025D  2025-05-03T21:48:42.670080    14.549882   
2  SAUDICOMSAT 2  2004-025E  2025-05-03T20:39:55.609056    14.495700   
3    APRIZESAT 1  2004-025G  2025-05-03T22:09:59.079744    14.514271   
4      NANOSAT-1  2004-049B  2025-05-04T01:39:49.540032    14.788376   

   ECCENTRICITY  INCLINATION  RA_OF_ASC_NODE  ARG_OF_PERICENTER  MEAN_ANOMALY  \
0      0.010690      98.0771        308.9818            45.9822      315.0114   
1      0.003204      98.5688        118.2715           194.7574      165.2690   
2      0.005695      98.6592        119.6961            59.8598      300.8221   
3      0.004405      98.6230        118.7079           320.6242       39.1752   
4      0.000465      98.1743        247.5181           153.3199      206.8251   

   EPHEMERIS_TYPE C

In [25]:
import pandas as pd
import numpy as np

# 输入文件：已包含 x,y,z, vx,vy,vz 的状态向量
input_csv = 'last_30_days_state_vectors_corrected.csv'
# 输出文件：将多出 v_r_km_s, v_t_km_s 两列
output_csv = 'state_vectors_rt.csv'

# 1. 读取数据
df = pd.read_csv(input_csv)

# 2. 计算径向和切向速度
def compute_rt_velocities(row):
    # 位置向量
    r = np.array([row['x_km'], row['y_km'], row['z_km']])
    # 速度向量
    v = np.array([row['vx_km_s'], row['vy_km_s'], row['vz_km_s']])
    # 归一化的径向单位向量
    r_hat = r / np.linalg.norm(r)
    # 径向速度 = v · r_hat
    v_r = np.dot(v, r_hat)
    # 切向速度向量 = v - v_r * r_hat
    v_t_vec = v - v_r * r_hat
    # 切向速度大小
    v_t = np.linalg.norm(v_t_vec)
    return pd.Series({'v_r_km_s': v_r, 'v_t_km_s': v_t})

# 对每行应用，并水平拼接新列
rt = df.apply(compute_rt_velocities, axis=1)
df = pd.concat([df, rt], axis=1)

# 3. 保存结果
df.to_csv(output_csv, index=False)

print(f'已生成包含径向/切向速度的文件：{output_csv}')


已生成包含径向/切向速度的文件：state_vectors_rt.csv
